In [1]:
import nltk
import urllib
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import re
import import_ipynb
from blog_class import Blog
import pandas as pd
from datetime import datetime, date, time

importing Jupyter notebook from blog_class.ipynb


In [2]:
def get_all_blogs(blog_page):
        soup = BeautifulSoup(urllib.urlopen(blog_page).read(), 'html.parser')
        blogs = [Blog(x.find('a')['href']) for x in soup.find_all("h2", {"class": "post-listing-simple"})]
        return blogs

In [3]:
# get all blogs and construct dataframe of independent variables
blog_page = 'https://blog.aurorasolar.com/all'
blogs = get_all_blogs(blog_page)
blog_df = pd.DataFrame({'title': [x.title() for x in blogs], 
                        'body_word_count': [x.body_word_count()[0] for x in blogs],
                        'author': [x.author() for x in blogs],
                        'post_date': [x.post_date() for x in blogs],
                        'url': [x.url for x in blogs],
                        'sub_heading_paragraph_ratio': [x.sub_heading_paragraph_ratio() for x in blogs],
                        'avg_word_count_in_sentence': [x.avg_word_count_in_sentence() for x in blogs],
                        'avg_word_count_paragraph': [x.avg_word_count_paragraph() for x in blogs],
                        'graphic_paragraph_ratio': [x.graphic_paragraph_ratio() for x in blogs],
                        'link_paragraph_ratio': [x.link_paragraph_ratio() for x in blogs],
                        'noun_pct': [x.word_type_percentage_body('noun') for x in blogs],
                        'adj_pct': [x.word_type_percentage_body('adj') for x in blogs],
                        'verb_pct': [x.word_type_percentage_body('verb') for x in blogs],
                        'adv_pct': [x.word_type_percentage_body('adv') for x in blogs],
                        'question_in_titles': [x.question_in_titles() for x in blogs],
                        'reading_level_1': [x.reading_level()[0] for x in blogs],
                        'reading_level_2': [x.reading_level()[1] for x in blogs]
                       })
# to make sure that the post date column has the type date
blog_df.post_date = pd.to_datetime(blog_df.post_date)
#save the file to be csv because it takes so much time to run
blog_df.to_csv('blogs_updated.csv', encoding='utf-8')
blog_df

In [7]:
# clean up google analytics, and construct dataframe of dependent variables

def format_url(txt):
    # format each url so that the url actually links to a webpage
    if txt == '/':
        return 'https://www.aurorasolar.com/'
    else:
        if 'https://' not in txt:
            txt = 'https://' + txt
        return re.sub('^/', 'https://blog.aurorasolar.com/', txt)

def get_title_for_google(url):
    try:
        if ('tag' not in url) and url != 'https://www.aurorasolar.com/':
            return re.sub(';', '', Blog(url).title())
        else:
            return 'Not Blog'
    except:
        return 'Page not found'
    
def clean_numeric(d):
    return int(re.sub('\D', '', d))

def clean_time(d):
    try:
        return sum([int(x) * y for x, y in zip(d.split(':'), [60**2, 60, 1])])
    except:
        return 0
    
def clean_percentage(d):
    return float(re.sub('%', '', d))/100

def aggregate(dt):
    # aggreate the stats of the blogs sharing the same titiles in different urls 
    new = dt
    dt['Pageviews'] = dt['Pageviews'].apply(clean_numeric)
    dt['sum_Pageviews'] = sum(dt['Pageviews'])
    dt['sum_Unique_Pageviews'] = sum(dt['Unique Pageviews'].apply(clean_numeric))
    dt['sum_Entrances'] = sum(dt['Entrances'].apply(clean_numeric))
    dt['TimeOnPage'] = round(float(sum(dt['Avg. Time on Page'].apply(clean_time) * dt['Pageviews']))/dt.iloc[0,:].sum_Pageviews, 1)
    dt['BounceRate'] = round(float(sum(dt['% Exit'].apply(clean_percentage) * dt['Pageviews']))/dt.iloc[0,:].sum_Pageviews, 1)
    return dt[['blog_title', 'sum_Pageviews', 'sum_Unique_Pageviews', 'sum_Entrances', 'TimeOnPage', 'BounceRate']]
    
def clean_up_google_analytics_data(google_analytics_csv):
    ga = pd.read_csv(google_analytics_csv)
    google_urls = ga.Page.apply(format_url)
    ga.loc[:,'blog_title'] = google_urls.apply(get_title_for_google)
    ga = ga[~ga.blog_title.isin(['Page not found', '', 'Not Blog'])].reset_index(drop=True).drop_duplicates()
    dependent_variables = ga.groupby('blog_title').apply(aggregate).drop_duplicates()
    return dependent_variables

In [8]:
data = clean_up_google_analytics_data('ga.csv')
# ga.csv is a downloaded file from aurora google analytics blog stats session

In [9]:
# data
# this process takes time too, save it as csv file
# data.to_csv('dependent_variables.csv', encoding='utf-8')

,blog_title,sum_Pageviews,sum_Unique_Pageviews,sum_Entrances,TimeOnPage,BounceRate
0,"Shade Losses in PV Systems, and Techniques to ...",18008,10674,9835,103.1,0.5
1,SDG&E’s New Time of Use Rates: How Solar Savin...,11440,6018,5844,91.9,0.5
2,Choosing the Right Size Inverter for Your Sola...,8757,5070,4827,90.9,0.5
3,How to Size a PV System from an Electricity Bill,6631,3492,2539,104.7,0.4
4,California’s New Smart Inverter Requirements: ...,6804,3363,3216,61.7,0.5
5,The Beginner’s Guide to Solar Energy,6167,3844,2853,109.2,0.5
6,The Ultimate Guide to NEM 2.0: Non-Bypassable ...,4346,2348,2173,101.3,0.5
7,What Makes a Solar Sales Proposal Successful? ...,3777,2282,2004,87.7,0.5
8,5 Reasons Blockchain Is Game-Changing for Sola...,3498,1759,1604,50.6,0.5
9,How to Estimate a Homeowner’s Energy Load Profile,3199,1765,1642,80.3,0.5


In [10]:
# combine the data together.
reserach_data = pd.merge(data, blog_df, how= 'right', left_on= 'blog_title', right_on = 'title')
reserach_data

,blog_title,sum_Pageviews,sum_Unique_Pageviews,sum_Entrances,TimeOnPage,BounceRate,author,post_date,title
0,"Shade Losses in PV Systems, and Techniques to ...",18008,10674,9835,103.1,0.5,Christian Brown,2016-10-07,"Shade Losses in PV Systems, and Techniques to ..."
1,SDG&E’s New Time of Use Rates: How Solar Savin...,11440,6018,5844,91.9,0.5,Andrew Gong,2017-12-06,SDG&E’s New Time of Use Rates: How Solar Savin...
2,Choosing the Right Size Inverter for Your Sola...,8757,5070,4827,90.9,0.5,David Bromberg,2017-03-14,Choosing the Right Size Inverter for Your Sola...
3,How to Size a PV System from an Electricity Bill,6631,3492,2539,104.7,0.4,Christian Brown,2016-10-07,How to Size a PV System from an Electricity Bill
4,California’s New Smart Inverter Requirements: ...,6804,3363,3216,61.7,0.5,Gwen Brown,2017-11-08,California’s New Smart Inverter Requirements: ...
5,The Beginner’s Guide to Solar Energy,6167,3844,2853,109.2,0.5,Christian Brown,2016-10-07,The Beginner’s Guide to Solar Energy
6,The Ultimate Guide to NEM 2.0: Non-Bypassable ...,4346,2348,2173,101.3,0.5,Andrew Gong,2017-06-15,The Ultimate Guide to NEM 2.0: Non-Bypassable ...
7,What Makes a Solar Sales Proposal Successful? ...,3777,2282,2004,87.7,0.5,Gwen Brown,2017-10-18,What Makes a Solar Sales Proposal Successful? ...
8,5 Reasons Blockchain Is Game-Changing for Sola...,3498,1759,1604,50.6,0.5,Gwen Brown,2018-02-21,5 Reasons Blockchain Is Game-Changing for Sola...
9,How to Estimate a Homeowner’s Energy Load Profile,3199,1765,1642,80.3,0.5,Gwen Brown,2017-04-10,How to Estimate a Homeowner’s Energy Load Profile


In [ ]:
# output working_data.csv to be the working data for analysis and modeling,
# save the working_data to be in the google sheet: https://docs.google.com/spreadsheets/d/1DJbsjTALDd6annAwbyGrpiLg2iFFHQUN-XQ5Ap2A1RE/edit#gid=0
# use vlookup to combine the output of content categorical variables that done by gwen, which is also saved in the sheet
# export the matched data to be the final working data and use as an input in the analysis and modeling part.
# research_data.to_csv('working_data.csv')
